In [62]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


In [63]:
df=pd.read_csv(r"C:\Users\sarve\Desktop\credit_card_fruda\resampled_fraud_data.csv")
df.columns
df['is_fraud'].value_counts()

0    7506
1    6153
Name: is_fraud, dtype: int64

In [65]:


#  preprocessed and stored in 'df'
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost classifier without 'use_label_encoder' parameter
model = xgb.XGBClassifier(eval_metric='logloss')

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix: \n{cm}")


Accuracy: 0.9784040995607614
Confusion Matrix: 
[[1479   33]
 [  26 1194]]


In [66]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=cv, scoring='f1')
print("F1 scores:", scores)
print("Average F1:", scores.mean())


F1 scores: [0.97808442 0.97685749 0.97886179 0.97440065 0.97934386]
Average F1: 0.9775096418243668


In [74]:
from sklearn.metrics import classification_report

# Load the test dataset
test_df = pd.read_csv(r"C:\Users\sarve\Desktop\credit_card_fruda\processed_fraud_data_test.csv")

# Drop unnecessary column
#test_df = test_df.drop('unix_time', axis=1)

# Separate features and labels
X_test_real = test_df.drop('is_fraud', axis=1)
y_test_real = test_df['is_fraud']

# Align feature columns to match training data
X_test_real = X_test_real[X_train.columns]  # <- Ensures correct order

# Predict using the trained model
# y_pred_real = model.predict(X_test_real)
y_proba = model.predict_proba(X_test_real)[:, 1]  # Probability of class 1
y_pred_custom = (y_proba > 0.5).astype(int)       # Lowering threshold helps catch more frauds


# Evaluate the model
# accuracy = accuracy_score(y_test_real, y_pred_real)
# cm = confusion_matrix(y_test_real, y_pred_real)
# report = classification_report(y_test_real, y_pred_real)
accuracy = accuracy_score(y_test_real, y_pred_custom)
cm = confusion_matrix(y_test_real, y_pred_custom)
report = classification_report(y_test_real, y_pred_custom)

# Display results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)



Accuracy: 0.9793942622080584
Confusion Matrix:
 [[542491  11083]
 [   368   1777]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99    553574
           1       0.14      0.83      0.24      2145

    accuracy                           0.98    555719
   macro avg       0.57      0.90      0.61    555719
weighted avg       1.00      0.98      0.99    555719

